In [3]:
:dep aoc2023 = { path = "../../" }
:dep anyhow = { version = "1.0.75" }

In [4]:
use aoc2023::utils::util;
use aoc2023::utils::grid3d::Coord3d;
use aoc2023::utils::grid::{Grid, Coord};
use std::{collections::{HashMap, HashSet}, mem::swap, str::FromStr};
use anyhow::Error;

In [5]:
let lines = util::lines_in("./input1");

In [6]:
#[derive(Debug)]
struct Brick {
    coords: Vec<Coord3d>,
}

impl FromStr for Brick {
    type Err = Error;

    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let (start_str, end_str) = s.split_once("~").ok_or(Error::msg("parse error"))?;
        let start_coord = Coord3d::from_str(start_str)?;
        let end_coord = Coord3d::from_str(end_str)?;

        Ok(Brick {
            coords: start_coord.coords_till(&end_coord),
        })
    }
}

impl Brick {
    fn get_num(&self, coords: &HashMap<Coord3d, usize>) -> usize {
        *coords.get(&self.coords[0]).unwrap()
    }

    fn can_go_down(&self, coords: &HashMap<Coord3d, usize>) -> bool {
        let num = self.get_num(coords);
        self.coords.iter().all(|coord| match coord.below() {
            None => false,
            Some(coord_below) => match coords.get(&coord_below) {
                None => true,
                Some(brick_num) => *brick_num == num,
            },
        })
    }

    fn go_down(&mut self, coords: &mut HashMap<Coord3d, usize>) {
        let num = self.get_num(coords);
        self.coords.iter().for_each(|coord| {
            coords.remove(coord).unwrap();
        });
        self.coords = self
            .coords
            .iter()
            .map(|coord| coord.below().unwrap())
            .collect();
        self.coords.iter().for_each(|coord| {
            assert!(coords.get(coord).is_none());
            coords.insert(*coord, num);
        });
    }

    fn supported_by(&self, coords: &HashMap<Coord3d, usize>) -> HashSet<usize> {
        let num = self.get_num(coords);
        self.coords
            .iter()
            .filter_map(|coord| coord.below())
            .filter_map(|coord_below| coords.get(&coord_below).cloned())
            // For vertically aligned bricks
            .filter(|box_num| *box_num != num)
            .collect::<HashSet<_>>()
    }
}

In [7]:
fn part1(lines: &Vec<String>) -> String {
    let mut bricks = lines
        .iter()
        .map(|line| Brick::from_str(line.as_str()).unwrap())
        .collect::<Vec<_>>();

    let mut coords: HashMap<Coord3d, usize> = bricks
        .iter()
        .enumerate()
        .flat_map(|(n, brick)| {
            brick
                .coords
                .iter()
                .map(|coord| (*coord, n))
                .collect::<Vec<_>>()
        })
        .collect();

    let mut run = true;
    while run {
        run = bricks.iter_mut().fold(false, |mut run, brick| {
            if brick.can_go_down(&coords) {
                brick.go_down(&mut coords);
                return true;
            }
            run
        });
    }

    let supporting_bricks = bricks
        .iter()
        .map(|brick| {
            let mut supported_by = brick.supported_by(&coords);
            if supported_by.len() == 1 {
                Some(supported_by.into_iter().next().unwrap())
            } else {
                None
            }
        })
        .filter(|s| s.is_some())
        .map(|s| s.unwrap())
        .collect::<HashSet<usize>>();

    (bricks.len() - supporting_bricks.len()).to_string()
}

In [8]:
part1(&lines)

"463"

In [9]:
let mut bricks = lines
        .iter()
        .map(|line| Brick::from_str(line.as_str()).unwrap())
        .collect::<Vec<_>>();

In [10]:
bricks.len()

1370

In [11]:
let mut coords: HashMap<Coord3d, usize> = bricks
        .iter()
        .enumerate()
        .flat_map(|(n, brick)| {
            brick
                .coords
                .iter()
                .map(|coord| (*coord, n))
                .collect::<Vec<_>>()
        })
        .collect();


In [12]:
coords.len()

4125

In [13]:
{
    let mut keys = coords.keys().collect::<Vec<&Coord3d>>();
    keys.sort_by(|ca, cb| if ca.r != cb.r {
        ca.r.cmp(&cb.r)
    } else if ca.p != cb.p {
        ca.p.cmp(&cb.p)
    } else {
        ca.q.cmp(&cb.q)
    });
    for key in keys[0..20].iter() {
        println!("{:?}, brick: {}", key, coords.get(key).unwrap());
    }
}

Coord3d { p: 0, q: 2, r: 1 }, brick: 655
Coord3d { p: 0, q: 3, r: 1 }, brick: 655
Coord3d { p: 0, q: 4, r: 1 }, brick: 655
Coord3d { p: 0, q: 5, r: 1 }, brick: 655
Coord3d { p: 1, q: 5, r: 1 }, brick: 649
Coord3d { p: 1, q: 6, r: 1 }, brick: 158
Coord3d { p: 1, q: 7, r: 1 }, brick: 158
Coord3d { p: 2, q: 3, r: 1 }, brick: 1215
Coord3d { p: 3, q: 3, r: 1 }, brick: 1215
Coord3d { p: 4, q: 3, r: 1 }, brick: 1215
Coord3d { p: 4, q: 9, r: 1 }, brick: 197
Coord3d { p: 5, q: 3, r: 1 }, brick: 1215
Coord3d { p: 5, q: 9, r: 1 }, brick: 197
Coord3d { p: 6, q: 9, r: 1 }, brick: 197
Coord3d { p: 7, q: 0, r: 1 }, brick: 1345
Coord3d { p: 8, q: 0, r: 1 }, brick: 1345
Coord3d { p: 9, q: 2, r: 1 }, brick: 288
Coord3d { p: 9, q: 3, r: 1 }, brick: 288
Coord3d { p: 9, q: 9, r: 1 }, brick: 1018
Coord3d { p: 0, q: 6, r: 2 }, brick: 1122


()

In [14]:
let max_p = coords.keys().fold(0, |max, coord| std::cmp::max(max, coord.p));
let max_q = coords.keys().fold(0, |max, coord| std::cmp::max(max, coord.q));
let max_r = coords.keys().fold(0, |max, coord| std::cmp::max(max, coord.r));
(max_p, max_q, max_r)

(9, 9, 359)

In [15]:
let grid_x = (0..max_r + 1).map(|r| {
    (0..max_p + 1).map(|p| {
        let first = (0..max_q + 1).find_map(|q| {
            coords.get(&Coord3d::new(p, q, r))
        });
        match first {
            None => "<......>".to_string(),
            Some(brick_num) => format!("<.{:0>4}.>", brick_num.to_string())
        }
    }).collect::<Vec<String>>()
}).collect::<Vec<Vec<String>>>();
let grid_str = grid_x[0..10].iter().rev().map(|row| row.join("")).collect::<Vec<_>>().join("\n");
println!("{}", grid_str);

<......><......><......><.0361.><.0361.><.0361.><.0118.><......><......><......>
<......><.1170.><.1170.><.0686.><.0686.><.0686.><.0686.><.0791.><.0981.><......>
<......><......><......><......><......><.0234.><......><......><......><......>
<......><......><......><.0876.><......><.1212.><.0460.><......><......><......>
<.0004.><.0170.><.0175.><.0889.><.0589.><.1145.><.0460.><.1145.><.1001.><......>
<.0555.><.0649.><.0659.><.0889.><......><.0079.><.0460.><.1028.><......><.0590.>
<.0247.><.0649.><.0659.><.0889.><......><.0079.><.0460.><.0853.><.0853.><.0005.>
<.1122.><.0649.><.0295.><.1201.><.0429.><.0429.><.0506.><.0506.><.0506.><.1018.>
<.0655.><.0649.><.1215.><.1215.><.1215.><.1215.><.0197.><.1345.><.1345.><.0288.>
<......><......><......><......><......><......><......><......><......><......>


In [16]:
let grid_y = (0..max_r + 1).map(|r| {
    (0..max_q + 1).map(|q| {
        let first = (0..max_p + 1).find_map(|p| {
            coords.get(&Coord3d::new(p, q, r))
        });
        match first {
            None => "<......>".to_string(),
            Some(brick_num) => format!("<.{:0>4}.>", brick_num.to_string())
        }
    }).collect::<Vec<String>>()
}).collect::<Vec<Vec<String>>>();
let grid_str = grid_y[0..10].iter().rev().map(|row| row.join("")).collect::<Vec<_>>().join("\n");
println!("{}", grid_str);

<.0118.><.0118.><.0118.><......><.0600.><.0600.><.0600.><.0600.><.0361.><......>
<.0686.><......><......><.1170.><.0791.><......><.0981.><......><......><......>
<......><......><......><......><......><......><......><.0234.><.0234.><......>
<.0460.><......><......><......><.0876.><.0876.><.0876.><.1212.><.1212.><.1213.>
<.0460.><.0004.><.0004.><.0589.><.0175.><.0170.><.0170.><.0170.><.0200.><.0478.>
<.0460.><.0889.><.0590.><......><......><.0649.><.0134.><.0079.><.0215.><.0555.>
<.0247.><.0247.><.0251.><.0251.><.0077.><.0077.><.0077.><.0077.><.0215.><.1054.>
<......><......><.0506.><.0429.><......><.0649.><.1122.><.1122.><.1122.><.1314.>
<.1345.><......><.0655.><.0655.><.0655.><.0655.><.0158.><.0158.><......><.0197.>
<......><......><......><......><......><......><......><......><......><......>


In [18]:
{
    let brick = bricks.get(0959).unwrap();
    println!("{:?}, {}", brick.coords, brick.can_go_down(&coords));
    println!("{:?}", brick.coords);
    
}

[Coord3d { p: 7, q: 5, r: 38 }, Coord3d { p: 7, q: 6, r: 38 }, Coord3d { p: 7, q: 7, r: 38 }, Coord3d { p: 7, q: 8, r: 38 }], true
[Coord3d { p: 7, q: 5, r: 38 }, Coord3d { p: 7, q: 6, r: 38 }, Coord3d { p: 7, q: 7, r: 38 }, Coord3d { p: 7, q: 8, r: 38 }]


()

In [19]:
    let mut run = true;
    while run {
        run = bricks.iter_mut().fold(false, |mut run, brick| {
            if brick.can_go_down(&coords) {
                brick.go_down(&mut coords);
                return true;
            }
            run
        });
    }

()

In [20]:
let grid_x = (0..max_r + 1).map(|r| {
    (0..max_p + 1).map(|p| {
        let first = (0..max_q + 1).find_map(|q| {
            coords.get(&Coord3d::new(p, q, r))
        });
        match first {
            None => "<......>".to_string(),
            Some(brick_num) => format!("<.{:0>4}.>", brick_num.to_string())
        }
    }).collect::<Vec<String>>()
}).collect::<Vec<Vec<String>>>();
let grid_str = grid_x[0..10].iter().rev().map(|row| row.join("")).collect::<Vec<_>>().join("\n");
println!("{}", grid_str);

<......><......><.1196.><.1196.><.0420.><.0244.><.0244.><.0085.><.1350.><......>
<......><.1262.><.0059.><.1038.><.0809.><.0388.><.0388.><.0388.><.0388.><.0627.>
<......><.0588.><.1073.><.1073.><.0310.><.0643.><.0643.><.0643.><.1246.><.1246.>
<......><.0994.><......><.0386.><.0808.><.0763.><.0118.><.1249.><.1249.><.1249.>
<.1248.><.1248.><.1248.><.0686.><.0686.><.0686.><.0686.><......><.1118.><......>
<......><.1170.><.1170.><.1170.><.1170.><.1145.><.0460.><.1145.><.0981.><.0752.>
<......><.0649.><.0175.><.0889.><.0589.><.0160.><.0460.><.1028.><.1001.><.1018.>
<.0004.><.0649.><.0659.><.0889.><.0429.><.0429.><.0460.><.0852.><.0853.><.0590.>
<.0247.><.0649.><.1215.><.0889.><.1215.><.1215.><.0460.><.1345.><.1345.><.0288.>
<......><......><......><......><......><......><......><......><......><......>


In [21]:
fn print_layer(layer: u32, max_p: u32, max_q: u32, coords: &HashMap<Coord3d, usize>) {
    let grid_xy = (0..max_p + 1).map(|p| {
        (0..max_q + 1).map(|q| {
            match coords.get(&Coord3d::new(p, q, layer)) {
                None => "<......>".to_string(),
                Some(brick_num) => format!("<.{:0>4}.>", brick_num.to_string())
            }
        }).collect::<Vec<String>>()
    }).collect::<Vec<Vec<String>>>();
    let grid_str = grid_xy.iter().rev().map(|row| row.join("")).collect::<Vec<_>>().join("\n");
    println!("{}", grid_str);
}

print_layer(18, max_p, max_q, &coords);

<......><......><......><......><......><......><.1218.><......><......><......>
<......><......><......><......><.0140.><......><.1218.><......><......><......>
<......><......><......><.0645.><.0140.><.0008.><.1218.><......><......><......>
<......><......><......><.0645.><......><.0008.><......><......><......><......>
<......><......><......><.0645.><.0236.><.0008.><......><......><......><......>
<......><......><......><.0645.><.0236.><.0008.><.1327.><.1327.><.1327.><......>
<......><.0624.><.0624.><.0624.><......><.0558.><......><......><......><......>
<......><.0810.><......><......><......><.0558.><.0179.><......><......><......>
<......><.0810.><.1091.><......><......><.0558.><.0179.><......><......><......>
<......><......><.1091.><......><......><......><.0179.><......><......><......>


In [22]:
print_layer(19, max_p, max_q, &coords);

<......><......><......><.1205.><.1205.><.1205.><.1205.><......><......><......>
<......><......><......><......><......><......><.1245.><.1245.><.1245.><......>
<......><......><.1259.><.1259.><......><.0959.><.0959.><.0959.><.0959.><......>
<.0469.><.0469.><.0469.><.0469.><......><......><.0203.><......><.1159.><......>
<......><......><......><.0256.><......><......><.0203.><......><.1159.><......>
<......><......><......><.0256.><......><......><.0203.><......><.1159.><......>
<......><......><......><.0256.><......><......><......><......><......><......>
<......><......><......><.0256.><......><......><......><......><......><......>
<.0662.><.0662.><.0662.><.0662.><......><......><.0903.><.0903.><......><......>
<......><......><......><......><.0996.><.0996.><.0996.><.0996.><.0996.><......>


In [24]:
    let supporting_bricks = bricks
        .iter()
        .map(|brick| {
            let mut supported_by = brick.supported_by(&coords);
            if supported_by.len() == 1 {
                Some(supported_by.into_iter().next().unwrap())
            } else {
                None
            }
        })
        .filter(|s| s.is_some())
        .map(|s| s.unwrap())
        .collect::<HashSet<usize>>();

In [25]:
let mut supporting_bricks = supporting_bricks.into_iter().collect::<Vec<_>>();
supporting_bricks.sort();

supporting_bricks

[1, 3, 6, 7, 9, 10, 13, 14, 15, 17, 18, 20, 22, 23, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 44, 45, 46, 49, 51, 52, 53, 54, 55, 56, 59, 60, 63, 64, 65, 68, 69, 72, 73, 78, 80, 83, 84, 86, 87, 89, 90, 91, 92, 94, 95, 99, 100, 103, 104, 106, 109, 110, 111, 112, 113, 114, 117, 118, 119, 121, 122, 123, 125, 126, 129, 130, 131, 133, 134, 136, 139, 141, 142, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 156, 162, 163, 165, 167, 168, 169, 176, 178, 179, 180, 181, 182, 185, 186, 187, 189, 190, 191, 193, 194, 195, 197, 198, 199, 201, 202, 204, 205, 206, 207, 208, 209, 212, 213, 216, 218, 220, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 237, 239, 240, 242, 243, 245, 246, 248, 249, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 262, 263, 264, 267, 268, 270, 271, 273, 276, 277, 281, 282, 284, 285, 286, 287, 288, 290, 291, 292, 293, 294, 295, 297, 303, 304, 308, 309, 310, 313, 314, 316, 317, 318, 320, 321, 323, 327, 328, 329, 330, 331, 335, 336, 337, 340, 343, 345, 34